In [1]:
%load_ext autoreload
%autoreload 2

import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

In [23]:
import json
import time
import logging
# logging.basicConfig(level=logging.INFO)


from typing import List, Dict, Any, Tuple, Union
from tqdm import tqdm

from prompts import SIMPLE_ENGLISH_TRANS_PROMPT
from core.services.provider import creator
from core.services.utils import calculate_cost_gpt4_turbo

from dotenv import load_dotenv
load_dotenv()

text_model_defaults = {"model" : "gpt-4-1106-preview", "temperature" : 0.1, "response_format" : {"type": "json_object"}}

In [6]:

def translate_to_eng(input_text: str):

    if len(input_text) > 100000:
        raise ValueError("Input Text is too long. Should be less than 100000 characters")

    conversation = [{"role": "system", "content": SIMPLE_ENGLISH_TRANS_PROMPT}]

    user_prompt = f"""
    INPUT_TEXT:

    //input_text//

    {input_text}

    //input_text//
    
    OUTPUT:
    """

    user_message = {'role': 'user', "content": user_prompt}
    conversation.append(user_message)

    response = creator(
                    **text_model_defaults,
                    messages = conversation,
                    )

    output = json.loads(response.choices[0].message.content)
    total_usage = response.usage.model_dump()

    return {"output": output, "total_usage": total_usage}


In [21]:
with open(r"C:\Users\DELL\Documents\Curate\curate-v1\core\services\translation\test_news_hindi.txt", 'rb') as f:
    text = f.read().decode('utf-8')

print(len(text))

eng = translate_to_eng(text[:10000])
print(eng["output"]["text"])
print(eng["total_usage"].model_dump())
cost = calculate_cost_gpt4_turbo(eng["total_usage"].model_dump())
print(cost)
print(cost/10000)

83707


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Breaking News in Hindi: Read the main and latest news of November 9. Subscribe for unlimited article access. Stay updated with every news, download Android Hindi News apps, iOS Hindi News apps, and Amarujala Hindi News apps on your mobile. Get all India News in Hindi related to live update of politics, sports, entertainment, technology, and education etc. Stay updated with us for all breaking news from India News and more news in Hindi. Express your feedback. ABP News Top 10, Morning Bulletin: Start your morning with news from ABP News, read all the major news from the country and the world together. ABP News Top 10, Night's Major News: Read all the major news from the country and the world together on ABP News for the night of November 9, 2023. If you have missed important news in the day's hustle, read national and international news including business, Bollywood, sports, and gadgets with one click. NCP Crisis: '...found 20,000 such affidavits', Sharad Pawar's faction accuses Ajit gr

Our cost per million characters: $12.5

Google cloud cost: $20